<a href="https://colab.research.google.com/github/okechukwuchude/Automating-Medical-Coding/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import json
import stanza
import csv
import pandas as pd

# Build an English pipeline
stanza.download('en', package='mimic', processors={'ner': 'i2b2'}) # download English model
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'}) # initialize English neural pipeline

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading these customized packages for language: en (English)...
| Processor       | Package        |
------------------------------------
| tokenize        | mimic          |
| pos             | mimic_charlm   |
| lemma           | mimic_nocharlm |
| depparse        | mimic_charlm   |
| ner             | i2b2           |
| pretrain        | mimic          |
| forward_charlm  | mimic          |
| backward_charlm | mimic          |



INFO:stanza:Downloaded file to /root/stanza_resources/en/tokenize/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/pos/mimic_charlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/lemma/mimic_nocharlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/depparse/mimic_charlm.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/ner/i2b2.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/pretrain/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/forward_charlm/mimic.pt


INFO:stanza:Downloaded file to /root/stanza_resources/en/backward_charlm/mimic.pt
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | mimic          |
| pos       | mimic_charlm   |
| lemma     | mimic_nocharlm |
| depparse  | mimic_charlm   |
| ner       | i2b2           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
# Function to process text using Stanza
def process_text_with_stanza(text):
    doc = nlp(text)
    return doc

In [5]:
# Global variable to store all extracted texts
extracted_texts = []

In [6]:
def extract_info_from_json(json_file_path, hadm_id_set):
    global extracted_texts

    # Open the JSON file
    with open(json_file_path, 'r') as file:
        # Load JSON data from the file
        data = json.load(file)

        # Extract 'hadm_id' and 'comment' from the JSON data
        hadm_id = data.get('hadm_id', None)
        comment = data.get('comment', None)

        # If 'hadm_id' is not found, print a warning message
        if hadm_id is None:
            print(f"Warning: 'hadm_id' not found in {json_file_path}")
            return None

        # Add 'hadm_id' to the set
        hadm_id_set.add(hadm_id)

        # Extract 'notes' from the JSON data
        notes = data.get('notes', [])

        # Iterate through each note
        for note in notes:
            note_info = {}  # Initialize a dictionary to store note information

            # Add 'hadm_id' to the note information
            note_info['hadm_id'] = hadm_id

            # Extract information from the note
            note_info['note_id'] = note.get('note_id', None)
            note_info['category'] = note.get('category', None)
            note_info['description'] = note.get('description', None)

            # Extract annotations from the note
            annotations = note.get('annotations', [])
            annotations_info = []  # Initialize a list to store annotation information

            # Iterate through each annotation in the note
            for annotation in annotations:
                annotation_info = {}  # Initialize a dictionary to store annotation information

                # Extract information from the annotation
                annotation_info['begin'] = annotation.get('begin', None)
                annotation_info['end'] = annotation.get('end', None)
                annotation_info['code'] = annotation.get('code', None)
                annotation_info['code_system'] = annotation.get('code_system', None)
                annotation_info['description'] = annotation.get('description', None)
                annotation_info['type'] = annotation.get('type', None)
                annotation_info['covered_text'] = annotation.get('covered_text', None)

                annotations_info.append(annotation_info)  # Append annotation information to the list

            note_info['annotations'] = annotations_info  # Add annotations information to the note
            note_info['text'] = note.get('text', None)

            # Process text with Stanza
            if note_info['text']:
                processed_text = process_text_with_stanza(note_info['text'])
                note_info['processed_text'] = processed_text

                # Extract sentence information and generate labels
                sentence_info = []
                for sent in processed_text.sentences:
                    tokens = [word.text for word in sent.words]
                    labels = ['O'] * len(tokens)  # Initialize labels as 'O'

                    for annotation in note_info['annotations']:
                        begin = annotation['begin']
                        end = annotation['end']

                        word_indices = find_word_indices(sent.words, begin, end)

                        # Assign labels
                        if word_indices:
                            start_idx, end_idx = word_indices[0], word_indices[-1]
                            labels[start_idx] = 'B'
                            for idx in range(start_idx + 1, end_idx + 1):
                                labels[idx] = 'I'
                        # for idx in word_indices:
                        #     if sent.words[idx].start_char == begin:
                        #         labels[idx] = 'B'
                        #     elif sent.words[idx].start_char > begin and sent.words[idx].end_char < end:
                        #         labels[idx] = 'I'

                    sentence_info.append({
                        'sentence_id': sent.index,
                        'words': tokens,
                        'labels': labels
                    })

                    # Print the result for each sentence
                    print('Sentence:', ' '.join(tokens))
                    print('Labels:', ' '.join(labels))
                    print()

                note_info['sentence_info'] = sentence_info

            extracted_texts.append(note_info)  # Append note information to the global variable

#

In [7]:
def find_word_indices(words, begin, end):
    # Create an empty list to store the indices of words found within the given range
    word_indices = []

    # Iterate through each token in the list of tokens
    for idx, word in enumerate(words):
        # Check if the start character of the token matches the beginning of the range
        # or if the token spans the beginning of the range
        if word.start_char == begin or (word.start_char < begin and word.end_char > begin):
            # If it matches, add the index of the token to the list of word indices
            word_indices.append(idx)
        # Check if the end character of the token matches the end of the range
        # or if the token spans the end of the range
        if word.end_char == end or (word.start_char < end and word.end_char > end):
            # If it matches, add the index of the token to the list of word indices
            word_indices.append(idx)
            break

    # Return the list of word indices found within the given range
    return word_indices

In [8]:
def generate_labels(words, annotations, start_token_idx, end_token_idx):
    labels = ['O'] * len(words)

    for annotation in annotations:
        code = annotation['code']

        # # Update labels for the matched words
        # if start_token_idx is not None and end_token_idx is not None:
        #     labels[start_token_idx] = f'B-{code}'
        #     for idx in range(start_token_idx + 1, end_token_idx + 1):
        #         labels[idx] = f'I-{code}'

    return labels

In [9]:
# Function to search for JSON files in a given folder and its subfolders

def search_files(folder_path):
    # Set to store unique hadm_id values
    hadm_id_set = set()

    # Recursively search for JSON files in the folder and its subfolders
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.json'):
                # Construct the full path to the JSON file
                json_file_path = os.path.join(root, filename)
                print("Processing:", json_file_path)
                # Call extract_info_from_json function to extract information from the JSON file
                extract_info_from_json(json_file_path, hadm_id_set)
                # Print a separator after processing each file
                print("=" * 50)

    # Print the count of unique hadm_id values
    print("Total unique hadm_id count:", len(hadm_id_set))



In [10]:
# Specify the path to the main folder containing subfolders with JSON files
main_folder_path = '/content/drive/MyDrive/#medical coding/testing'

# Call the search_files function to start searching for JSON files in the main folder and its subfolders
search_files(main_folder_path)

Streaming output truncated to the last 5000 lines.

Sentence: Brief Hospital Course : # Lower GI Bleed : Admitted with a Hct of 29 from baseline 43 and orthostatic by vital signs .
Labels: O O O O O O O O O O O O O O O O O O O O O O O O

Sentence: She was aggressively volume resuscitated with 5 L of crystalloid and transfused 2 units of PRBCs after continuning to pass dilute blood with a Golytely prep , which was then held the first night of the hospitalization after completing half of the prep .
Labels: O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O

Sentence: On hospital day 2 , she underwent colonoscopy , which was remarkable for bleeding in the cecum , the site of 2 of her polypectomies 9 days prior to admission ; 4 clips were placed with adequate hemostasis .
Labels: O O O O O O O O O O O O O B I I I I I I I I I I I O O O O O O O O O O O O O O

Sentence: Her volume and hematocrit subsequently remained stable .
Labels: O O O O O O O O

Sente

In [11]:
import pandas as pd

# Function to create a DataFrame from the extracted sentence information
def create_dataframe(extracted_texts):
    data = []

    for note_info in extracted_texts:
        if 'sentence_info' in note_info:
            for sentence_info in note_info['sentence_info']:
                words = sentence_info['words']
                labels = sentence_info['labels']

                for word, label in zip(words, labels):
                    data.append({
                        'hadm_id': note_info['hadm_id'],
                        'note_id': note_info['note_id'],
                        'sentence_id': sentence_info['sentence_id'],
                        'word': word,
                        'label': label
                    })

    df = pd.DataFrame(data)
    return df

df = create_dataframe(extracted_texts)
print(df)

       hadm_id  note_id  sentence_id           word label
0       101525   566615            0   Demographics     O
1       101525   566615            0            Day     O
2       101525   566615            0             of     O
3       101525   566615            0     intubation     O
4       101525   566615            0              :     O
...        ...      ...          ...            ...   ...
40837   100463    55049          107            **]     O
40838   100463    55049          108      Completed     O
40839   100463    55049          108             by     O
40840   100463    55049          108  :[**2109-7-31     O
40841   100463    55049          108            **]     O

[40842 rows x 5 columns]


In [12]:
label_counts = df['label'].value_counts()
print(label_counts)

O    39926
I      582
B      334
Name: label, dtype: int64


In [13]:
token_df= df.drop(['hadm_id', 'note_id',], axis=1)

In [14]:
token_df

,sentence_id,word,label
0,0,Demographics,O
1,0,Day,O
2,0,of,O
3,0,intubation,O
4,0,:,O
...,...,...,...
40837,107,**],O
40838,108,Completed,O
40839,108,by,O
40840,108,:[**2109-7-31,O


In [15]:
token_df.rename(columns = {'word':'words', 'label':'labels'}, inplace = True)

In [16]:
token_df.head()

,sentence_id,words,labels
0,0,Demographics,O
1,0,Day,O
2,0,of,O
3,0,intubation,O
4,0,:,O


In [17]:
x = token_df[['sentence_id', 'words']]
y = token_df['labels']

SPLITTING DATA

In [18]:
# Splitting data into train and test sets with stratified sampling

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, stratify=y)

In [19]:
#build the training and test data
train_data = pd.DataFrame({'sentence_id': x_train['sentence_id'], 'words': x_train['words'], 'labels': y_train})
test_data = pd.DataFrame({'sentence_id': x_test['sentence_id'], 'words': x_test['words'], 'labels': y_test})

In [20]:
train_data

,sentence_id,words,labels
2303,9,mL,O
4117,9,:,O
34939,107,**],O
36457,113,home,O
24895,13,Cardiovascular,O
...,...,...,...
10738,17,following,O
17143,65,hepatomegaly,I
35965,82,W/,O
26132,0,),O


MODEL TRAINING

BERT

In [21]:
pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 

In [22]:
from simpletransformers.ner import NERModel, NERArgs

In [23]:
label = token_df['labels'].unique().tolist()
label

['O', 'B', 'I']

In [27]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [28]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args, use_cuda = True)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from sklearn.metrics import accuracy_score

model.train_model(train_data, eval_data = test_data, acc = accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.15437957166827151)

In [30]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
result

{'eval_loss': 0.05555096604595227,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

In [32]:
prediction, model_output = model.predict(['There is also intraventricular extension into the ipsilateral and contralateral lateral ventricles.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
prediction

[[{'There': 'O'},
  {'is': 'O'},
  {'also': 'O'},
  {'intraventricular': 'O'},
  {'extension': 'O'},
  {'into': 'O'},
  {'the': 'O'},
  {'ipsilateral': 'O'},
  {'and': 'O'},
  {'contralateral': 'O'},
  {'lateral': 'O'},
  {'ventricles.': 'O'}]]

BioBERT

In [34]:
bio_label = token_df['labels'].unique().tolist()
bio_label

['O', 'B', 'I']

In [35]:
from transformers import BertTokenizer, BertForMaskedLM, BertForTokenClassification
import torch

Converting tokens to token IDs and create attention masks

In [37]:
# Convert tokens to token IDs and create attention masks
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
x_train_tokenized = tokenizer(x_train['words'].tolist(), padding=True, truncation=True, return_tensors="pt")
x_test_tokenized = tokenizer(x_test['words'].tolist(), padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Convert labels to label IDs

In [38]:
# Convert labels to label IDs
label2id = {label: idx for idx, label in enumerate(df['label'].unique())}
y_train_ids = torch.tensor([label2id[label] for label in y_train.tolist()])
y_test_ids = torch.tensor([label2id[label] for label in y_test.tolist()])

CREATE DATALOADER

In [39]:
from torch.utils.data import TensorDataset, DataLoader

In [40]:
# Create DataLoader for training and validation
train_dataset = TensorDataset(x_train_tokenized['input_ids'],
                              x_train_tokenized['attention_mask'],
                              y_train_ids)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(x_test_tokenized['input_ids'],
                            x_test_tokenized['attention_mask'],
                            y_test_ids)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [42]:
# Define optimizer and loss function
model = BertForTokenClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.2', num_labels=len(label2id))
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
# Fine-tuning loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Training loop
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Evaluation loop
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print training and validation metrics
    train_loss = loss.item()
    val_loss /= len(val_loader)
    accuracy = correct / total
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {accuracy:.4f}")

Epoch 1/10


ValueError: Expected input batch_size (2560) to match target batch_size (32).